In [1]:
import json
import numpy as np

## Modelo Poisson 4 Parâmetros

In [ ]:
# Arquivos de entrada
probabilities_file = "probabilities.json"
years = [2021, 2022, 2023]
brasileirao_files = [f"dados/brasileirao_{year}.json" for year in years]

# Carregar o arquivo probabilities.json
with open(probabilities_file, "r") as f:
    probabilities = json.load(f)

# Combinar os resultados reais de todos os anos
real_results = {}

for file in brasileirao_files:
    with open(file, "r") as f:
        real_results.update(json.load(f))

# Função para converter o resultado real no formato esperado
def get_real_result(result):
    home_goals, away_goals = map(int, result.split(" X "))
    if home_goals > away_goals:
        return [1, 0, 0]  # Vitória do mandante
    elif home_goals == away_goals:
        return [0, 1, 0]  # Empate
    else:
        return [0, 0, 1]  # Vitória do visitante

# Calcular o Brier Score
def calculate_brier_score(probabilities, real):
    return sum((p - r) ** 2 for p, r in zip(probabilities, real))

# Listas para armazenar os Brier Scores
brier_scores_model = []
brier_scores_odds = []

# Processar cada jogo de teste nos três anos
for game_id, game_data in probabilities.items():
    # Obter as probabilidades do modelo e das odds
    model_probs = list(game_data["ModelProbabilities"].values())
    odds_probs = list(game_data["OddsProbabilities"].values())
    
    # Obter o resultado real
    year, game_number = game_id.split("_")
    real_result = get_real_result(real_results[game_number]["Result"])
    
    # Calcular os Brier Scores para o modelo e as odds
    model_brier = calculate_brier_score(model_probs, real_result)
    odds_brier = calculate_brier_score(odds_probs, real_result)
    
    # Armazenar os Brier Scores
    brier_scores_model.append(model_brier)
    brier_scores_odds.append(odds_brier)

# Calcular os Brier Scores gerais (médias)
brier_score_model_mean = np.mean(brier_scores_model)
brier_score_odds_mean = np.mean(brier_scores_odds)

# Exibir os resultados
print(f"Brier Score Geral do Modelo: {brier_score_model_mean:.4f}")
print(f"Brier Score Geral das Odds: {brier_score_odds_mean:.4f}")


Brier Score Geral do Modelo: 0.6048
Brier Score Geral das Odds: 0.5369


In [6]:
# Arquivos de entrada
probabilities_file = "probabilities.json"
years = [2021, 2022, 2023]
brasileirao_files = [f"dados/brasileirao_{year}.json" for year in years]

# Carregar o arquivo probabilities.json
with open(probabilities_file, "r") as f:
    probabilities = json.load(f)

# Combinar os resultados reais de todos os anos
real_results = {}

for file in brasileirao_files:
    with open(file, "r") as f:
        real_results.update(json.load(f))

# Função para converter o resultado real no formato esperado
def get_real_result(result):
    home_goals, away_goals = map(int, result.split(" X "))
    if home_goals > away_goals:
        return [1, 0, 0]  # Vitória do mandante
    elif home_goals == away_goals:
        return [0, 1, 0]  # Empate
    else:
        return [0, 0, 1]  # Vitória do visitante

# Função para calcular IGN10 (Medida de Ignorância)
def calculate_ign10(probabilities, real):
    ign10 = 0
    for p, r in zip(probabilities, real):
        if r == 1:  # Apenas a probabilidade do evento ocorrido é relevante
            ign10 = -np.log10(p) if p > 0 else float('inf')  # Evitar log(0)
    return ign10

# Listas para armazenar os valores de IGN10
ign10_scores_model = []
ign10_scores_odds = []

# Processar cada jogo de teste nos três anos
for game_id, game_data in probabilities.items():
    # Obter as probabilidades do modelo e das odds
    model_probs = list(game_data["ModelProbabilities"].values())
    odds_probs = list(game_data["OddsProbabilities"].values())
    
    # Obter o resultado real
    year, game_number = game_id.split("_")
    real_result = get_real_result(real_results[game_number]["Result"])
    
    # Calcular a IGN10 para o modelo e as odds
    model_ign10 = calculate_ign10(model_probs, real_result)
    odds_ign10 = calculate_ign10(odds_probs, real_result)
    
    # Armazenar os valores de IGN10
    ign10_scores_model.append(model_ign10)
    ign10_scores_odds.append(odds_ign10)

# Calcular os valores gerais de IGN10 (médias)
ign10_model_mean = np.mean(ign10_scores_model)
ign10_odds_mean = np.mean(ign10_scores_odds)

# Exibir os resultados
print(f"IGN10 Geral do Modelo: {ign10_model_mean:.4f}")
print(f"IGN10 Geral das Odds: {ign10_odds_mean:.4f}")


IGN10 Geral do Modelo: 0.4405
IGN10 Geral das Odds: 0.3950


In [2]:
# Arquivos de entrada
probabilities_file = "probabilities.json"
years = [2021, 2022, 2023]
brasileirao_files = [f"dados/brasileirao_{year}.json" for year in years]

# Carregar o arquivo probabilities.json
with open(probabilities_file, "r") as f:
    probabilities = json.load(f)

# Combinar os resultados reais de todos os anos
real_results = {}

for file in brasileirao_files:
    with open(file, "r") as f:
        real_results.update(json.load(f))

# Função para converter o resultado real no formato esperado
def get_real_result(result):
    home_goals, away_goals = map(int, result.split(" X "))
    if home_goals > away_goals:
        return [1, 0, 0]  # Vitória do mandante
    elif home_goals == away_goals:
        return [0, 1, 0]  # Empate
    else:
        return [0, 0, 1]  # Vitória do visitante

# Função para calcular o RPS (Ranked Probability Score)
def calculate_rps(probabilities, real):
    d = len(probabilities)  # Dimensão do espaço de eventos (d = 3 no nosso caso)
    cumulative_sum_probs = np.cumsum(probabilities)
    cumulative_sum_real = np.cumsum(real)
    rps = sum((cumulative_sum_probs[i] - cumulative_sum_real[i]) ** 2 for i in range(d - 1))
    return rps / (d - 1)

# Listas para armazenar os valores de RPS
rps_scores_model = []
rps_scores_odds = []

# Processar cada jogo de teste nos três anos
for game_id, game_data in probabilities.items():
    # Obter as probabilidades do modelo e das odds
    model_probs = list(game_data["ModelProbabilities"].values())
    odds_probs = list(game_data["OddsProbabilities"].values())
    
    # Obter o resultado real
    year, game_number = game_id.split("_")
    real_result = get_real_result(real_results[game_number]["Result"])
    
    # Calcular o RPS para o modelo e as odds
    model_rps = calculate_rps(model_probs, real_result)
    odds_rps = calculate_rps(odds_probs, real_result)
    
    # Armazenar os valores de RPS
    rps_scores_model.append(model_rps)
    rps_scores_odds.append(odds_rps)

# Calcular os valores gerais de RPS (médias)
rps_model_mean = np.mean(rps_scores_model)
rps_odds_mean = np.mean(rps_scores_odds)

# Exibir os resultados
print(f"RPS Geral do Modelo: {rps_model_mean:.4f}")
print(f"RPS Geral das Odds: {rps_odds_mean:.4f}")


RPS Geral do Modelo: 0.2205
RPS Geral das Odds: 0.1868
